In [1]:
import __init__
import vsrl_utils as vu
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

added C:\Users\antek\Desktop\v-coco\coco\PythonAPI to pythonpath


In [2]:
def draw_bbox(plt, ax, rois, fill=False, linewidth=2, edgecolor=[1.0, 0.0, 0.0], **kwargs):
    for i in range(rois.shape[0]):
        roi = rois[i,:].astype(int)
        ax.add_patch(plt.Rectangle((roi[0], roi[1]),
            roi[2] - roi[0], roi[3] - roi[1],
            fill=False, linewidth=linewidth, edgecolor=edgecolor, **kwargs))

def subplot(plt, xxx_todo_changeme, xxx_todo_changeme1 = (10, 10)):
    (Y, X) = xxx_todo_changeme
    (sz_y, sz_x) = xxx_todo_changeme1
    plt.rcParams['figure.figsize'] = (X*sz_x, Y*sz_y)
    fig, axes = plt.subplots(Y, X)
    return fig, axes


In [3]:
# Load COCO annotations for V-COCO images
coco = vu.load_coco()

# Load the VCOCO annotations for vcoco_train image set
vcoco_all = vu.load_vcoco('vcoco_train')
for x in vcoco_all:
    x = vu.attach_gt_boxes(x, coco)

loading annotations into memory...
Done (t=3.31s)
creating index...
index created!


In [4]:
# Action classes and roles in V-COCO
classes = [x['action_name'] for x in vcoco_all]
for i, x in enumerate(vcoco_all):
    print('{:>20s}'.format(x['action_name']), x['role_name'])


                hold ['agent', 'obj']
               stand ['agent']
                 sit ['agent', 'instr']
                ride ['agent', 'instr']
                walk ['agent']
                look ['agent', 'obj']
                 hit ['agent', 'instr', 'obj']
                 eat ['agent', 'obj', 'instr']
                jump ['agent', 'instr']
                 lay ['agent', 'instr']
       talk_on_phone ['agent', 'instr']
               carry ['agent', 'obj']
               throw ['agent', 'obj']
               catch ['agent', 'obj']
                 cut ['agent', 'instr', 'obj']
                 run ['agent']
    work_on_computer ['agent', 'instr']
                 ski ['agent', 'instr']
                surf ['agent', 'instr']
          skateboard ['agent', 'instr']
               smile ['agent']
               drink ['agent', 'instr']
                kick ['agent', 'obj']
               point ['agent', 'instr']
                read ['agent', 'obj']
           snowboard ['agent'

In [5]:
labels = [x['label'] for x in vcoco_all]

# the demo here laods images from the COCO website, 
# you can alternatively use your own local folder of COCO images.
load_coco_image_from_web = True
if load_coco_image_from_web:
    from PIL import Image
    import requests
    from io import BytesIO

In [6]:
import ipywidgets as widgets
from ipywidgets import Layout, HBox
from IPython.display import display

inst_index = 0
all_inst = []
index = 0
positive_index = []


out = widgets.Output()

action = widgets.SelectMultiple(
    options = classes,
    value = ['hold', 'stand', 'sit', 'ride', 'walk', 'look', 'hit', 'eat', 'jump', 'lay', 'talk_on_phone', 'carry', 'throw', 'catch', 'cut', 'run', 'work_on_computer', 'ski', 'surf', 'skateboard', 'smile', 'drink', 'kick', 'point', 'read', 'snowboard'],
    description='action_name',
    disabled=False,
    layout = Layout(width='50%', height='160px', display='flex')
)

all_on_one_img = widgets.Checkbox(
    value=False,
    description='All actions on one instance',
    disabled=False,
    indent=False
)

search_button = widgets.Button(
    value=False,
    description='Search',
    disabled=False,
    layout=Layout(width='20%', height='40px')
)

@out.capture()
def search_button_click(b):
    out.clear_output()
    global positive_index
    global index
    index = 0
    positive_index  = list(search(list(action.value), all_on_one_img.value))
    if len(positive_index) > 0:
        print(str(len(positive_index)) + ' results.')
        get_all_instances()
        display_img(positive_index[0])
    else:
        print('No results.')
    
search_button.on_click(search_button_click)

next_instance_button = widgets.Button(
    value=False,
    description='Next human instace',
    disabled=False,
    layout=Layout(width='20%', height='40px')
)

@out.capture()
def next_inst_button_click(b):
    out.clear_output()
    global all_inst
    if len(all_inst) == 0:
        return
    global inst_index
    inst_index = inst_index + 1
    if inst_index >= len(all_inst):
        inst_index = 0
    display_img(all_inst[inst_index])

next_instance_button.on_click(next_inst_button_click)

prev_instance_button = widgets.Button(
    value=False,
    description='Previous human instance',
    disabled=False,
    layout=Layout(width='20%', height='40px')
)

@out.capture()
def prev_inst_button_click(b):
    out.clear_output()
    global all_inst
    if len(all_inst) == 0:
        return
    global inst_index
    inst_index = inst_index - 1
    if inst_index < 0:
        inst_index = len(all_inst) - 1
    display_img(all_inst[inst_index])

prev_instance_button.on_click(prev_inst_button_click)

next_img_button = widgets.Button(
    value=False,
    description='Next image',
    disabled=False,
    layout=Layout(width='20%', height='40px')
)

@out.capture()
def next_img_button_click(b):
    out.clear_output()
    global positive_index
    if len(positive_index) == 0:
        return
    global index
    index = index + 1
    if index >= len(positive_index):
        index = 0        
    get_all_instances()
    display_img(positive_index[index])

next_img_button.on_click(next_img_button_click)

prev_img_button = widgets.Button(
    value=False,
    description='Previous image',
    disabled=False,
    layout=Layout(width='20%', height='40px')
)

@out.capture()
def prev_img_button_click(b):
    out.clear_output()
    global positive_index
    if len(positive_index) == 0:
        return
    global index
    index = index - 1
    if index < 0:
        index = len(positive_index) - 1  
    get_all_instances()
    display_img(positive_index[index])
    

prev_img_button.on_click(prev_img_button_click)

@out.capture()
def get_all_instances():
    global inst_index
    inst_index = 0
    global all_inst
    img_id = [vcoco_all[0]['image_id'][int(positive_index[index])][0]]
    all_inst  = search_by_id(img_id[0])
    print('Instances on image: ' + str(len(all_inst)))
    

In [7]:

def search(classes_list, all_on_one_img):
    classes = [x['action_name'] for x in vcoco_all]
    res = []
    img_ids = []
    for x in classes_list:
        cls_id = classes.index(x)
        vcoco = vcoco_all[cls_id]
        positive_index = np.where(vcoco['label'] == 1)[0]       
        if all_on_one_img:
            if len(img_ids) == 0:
                img_ids = positive_index
            else:
                ids_copy = img_ids.copy()
                img_ids = []
                for img_id in positive_index:
                    if img_id in ids_copy:
                        img_ids = np.append(img_ids, img_id)
        else:
            img_ids = np.append(img_ids, positive_index) 
    res = img_ids
    res = list(np.unique(res))
    return res
 
def search_by_id(img_id):
    img_id = int(img_id)
    res = np.where(vcoco_all[0]['image_id'] == img_id)[0]
    return res

@out.capture()
def display_img(id):
    
    id = int(id)
    cc = plt.get_cmap('hsv', lut=4)
    print('Image_id: ' + str([vcoco_all[0]['image_id'][id][0]]))
    print('')
    
    actions = []
    
    for j in range(26):
        if labels[j][id] == 1:
            print('Action: ' + classes[j] + ' ')           
            actions = np.append(actions, classes[j])
            print('Role name: ' + ', '.join(vcoco_all[j]['role_name']))      
            
            for k in range(len(vcoco_all[j]['role_name'])):
                if len(vcoco_all[j]['include'][k]):
                    print('Possible ' + vcoco_all[j]['role_name'][k] + ':')
                    print(vcoco_all[j]['include'][k])
            print('')
    #print('Instance\'s actions: ' + ', '.join(actions))
    # load image
    coco_image = coco.loadImgs(ids=[vcoco_all[0]['image_id'][id][0]])[0]

    if load_coco_image_from_web:
        coco_url = coco_image.get('coco_url')
        response = requests.get(coco_url)
        im = Image.open(BytesIO(response.content))           
        
    with out:
        sy = 4.; sx = float(np.shape(im)[1])/float(np.shape(im)[0])*sy;
        fig, ax = subplot(plt, (1,1), (sy,sx)); ax.set_axis_off(); 

        ax.imshow(im)

        # draw bounding box for agent
        draw_bbox(plt, ax, vcoco_all[0]['bbox'][[id],:], edgecolor=cc(0)[:3])
        role_bbox = vcoco_all[0]['role_bbox'][id,:]*1.
        role_bbox = role_bbox.reshape((-1,4))
        for j in range(1, len(vcoco_all[0]['role_name'])):
            if not np.isnan(role_bbox[j,0]):
                draw_bbox(plt, ax, role_bbox[[j],:], edgecolor=cc(j)[:3])
        plt.show(fig)

In [8]:
display(HBox([action, all_on_one_img]))
display(HBox([next_img_button, prev_img_button, next_instance_button, prev_instance_button]))
display(search_button)

display(out)

Button(description='Search', layout=Layout(height='40px', width='20%'), style=ButtonStyle())

Output()